**REPASO 2 30/10/2021 - Procedimientos Almacenados Avanzados**

In [69]:
ALTER TABLE PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO
ADD MODIFICADO_SN BIT NOT NULL DEFAULT 0,
    ELIMINADO_SN BIT NOT NULL DEFAULT 0;

: Msg 2705, Level 16, State 4, Line 1
Los nombres de columna de cada tabla deben ser únicos. 'MODIFICADO_SN' de la tabla 'PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO' se especifica más de una vez.

Total execution time: 00:00:00.089

In [15]:
SELECT TOP 10 * FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO;
SELECT COUNT(*) AS ICR FROM PRUEBAS_ALUMNOS.dbo.ICR_FACT_COSTE_DEPARTAMENTO;
SELECT COUNT(*) AS AFR FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO;

(10 rows affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.107

COMERCIAL_ID,MES_ID,DEPARTAMENTO_ID,DEPARTAMENTO_DESC,CANTIDAD_HORAS,COSTE_DEPARTAMENTO,MODIFICADO_SN,ELIMINADO_SN
COMERCIAL 115,201904,1,VENTAS,NULL,274.46,0,0
COMERCIAL 115,201904,2,MARKETING,NULL,209.88,0,0
COMERCIAL 115,201904,3,RRHH,NULL,265.23,0,0
COMERCIAL 115,201904,4,LOGISTICA,NULL,242.17,0,0
COMERCIAL 115,201904,5,FINANCIERO,NULL,205.27,0,0
COMERCIAL 115,201905,1,VENTAS,NULL,275.31,0,0
COMERCIAL 115,201905,2,MARKETING,NULL,179.55,0,0
COMERCIAL 115,201905,3,RRHH,NULL,265.73,0,0
COMERCIAL 115,201905,4,LOGISTICA,NULL,292.07,0,0
COMERCIAL 115,201905,5,FINANCIERO,NULL,184.34,0,0


ICR
390


AFR
270


In [16]:
SELECT TOP 1 * FROM PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO

(1 row affected)

Total execution time: 00:00:00.115

COMERCIAL_ID,MES_ID,DEPARTAMENTO_ID,DEPARTAMENTO_DESC,CANTIDAD_HORAS,COSTE_DEPARTAMENTO,MODIFICADO_SN,ELIMINADO_SN
COMERCIAL 115,201904,1,VENTAS,NULL,274.46,0,0


In [17]:
USE PRUEBAS_ALUMNOS;
GO

CREATE OR ALTER PROCEDURE dbo.PR_AFR_REPARTO_COSTE_DPTO_2 --DECLARAMOS PROCEDIMIENTO
    @p_mes_inicial INTEGER, --PARAMETRO 1: MES INICIAL
    @p_mes_final INTEGER --PARAMETRO 2: MES FINAL
AS 
BEGIN
    DECLARE @ORIGEN TABLE (
        COMERCIAL_ID VARCHAR(50) NOT NULL,
        MES_ID INT NOT NULL,
        DEPARTAMENTO_ID INT NOT NULL,
        DEPARTAMENTO_DESC VARCHAR(15),
        COSTE_DEPARTAMENTO DECIMAL(15,2)
    );  --DECLARAMOS COMO VARIABLE UNA TABLA NUEVA CON LOS CAMPOS QUE QUEREMOS. AHÍ SE INSERTARÁN DATOS EN EL PROCEDIMIENTO
        
    -- Primero calculamos el total de horas por comercial mes
    WITH COMERCIAL_HORAS_MES AS (
        SELECT FCH.COMERCIAL_ID,
            FCH.MES_ID,
            SUM(FCH.CANTIDAD_HORAS) TOTAL_HORAS_COMERCIAL_MES
        FROM nSLDina.dbo.FACT_COMERCIAL_HORAS FCH
        GROUP BY FCH.COMERCIAL_ID,
                FCH.MES_ID
        ),

    -- Ahora que tenemos el total de horas de cada comercial por mes, calculamos el porcentaje de horas que corresponde a 
    -- cada departamente por comercial y mes
    PCT_COMERCIAL_DPTO_MES AS (
        SELECT FCH.COMERCIAL_ID,
            FCH.MES_ID,
            FCH.DEPARTAMENTO_ID,
            FCH.CANTIDAD_HORAS / CAST(TOTAL_HORAS_COMERCIAL_MES AS NUMERIC(5,2)) AS PCT_DEPARTAMENTO
        FROM nSLDina.dbo.FACT_COMERCIAL_HORAS FCH
            INNER JOIN COMERCIAL_HORAS_MES CHM
                ON FCH.COMERCIAL_ID = CHM.COMERCIAL_ID AND
                    FCH.MES_ID = CHM.MES_ID
        )

    INSERT INTO @ORIGEN (
            COMERCIAL_ID,
            MES_ID,
            DEPARTAMENTO_ID,
            DEPARTAMENTO_DESC,
            COSTE_DEPARTAMENTO) -- SE INSERTAN LOS DATOS EN NUESTRA TABLA CREADA

    SELECT PCDM.COMERCIAL_ID,
            PCDM.MES_ID,
            PCDM.DEPARTAMENTO_ID,
            DD.DEPARTAMENTO_DESC,
            ROUND(FN.IMPORTE_NOMINA * PCDM.PCT_DEPARTAMENTO, 2) AS COSTE_DEPARTAMENTO
    FROM nSLDina.dbo.FACT_NOMINA FN 
        INNER JOIN PCT_COMERCIAL_DPTO_MES PCDM
            ON FN.EMPLEADO_ID = PCDM.COMERCIAL_ID AND 
                FN.MES_ID = PCDM.MES_ID
        INNER JOIN nSLDina.dbo.DIM_DEPARTAMENTO DD
            ON PCDM.DEPARTAMENTO_ID = DD.DEPARTAMENTO_ID
    WHERE PCDM.MES_ID BETWEEN @p_mes_inicial AND @p_mes_final;

    BEGIN TRY --HACEMOS EL TRY PARA PROTEGERNOS ANTE ERRORES
        BEGIN TRAN; --COMENZAMOS LA TRANSACCION, QUE SERÁ UNA SENTENCIA MERGE EN LA QUE UNIREMOS LA TABLA @ORIGEN CON NUESTRA TABLA VIEJA, CON LOS DATOS ANTIGUOS (AFR)

        /* INTRODUCIR AQUÍ LA SENTENCIA MERGE */
        MERGE INTO PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO AS AFR
            USING @ORIGEN AS O
                ON O.COMERCIAL_ID = AFR.COMERCIAL_ID AND O.MES_ID = AFR.MES_ID AND O.DEPARTAMENTO_ID = AFR.DEPARTAMENTO_ID
            WHEN MATCHED THEN --LO QUE YA ESTA EN LOS DOS SITIOS LO ACTUALIZO
                UPDATE SET 
                    AFR.DEPARTAMENTO_DESC = O.DEPARTAMENTO_DESC,
                    AFR.COSTE_DEPARTAMENTO = O.DEPARTAMENTO_DESC,
                    AFR.MODIFICADO_SN = 1,
                    AFR.ELIMINADO_SN = 0
            WHEN NOT MATCHED BY TARGET THEN -- LO QUE NO ESTÁ EN LA TABLA AFR LO INSERTO
                INSERT 
                (
                    COMERCIAL_ID,
                    MES_ID,
                    DEPARTAMENTO_ID,
                    DEPARTAMENTO_DESC,
                    COSTE_DEPARTAMENTO
                )

                VALUES(
                    O.COMERCIAL_ID,
                    O.MES_ID,
                    O.DEPARTAMENTO_ID,
                    O.DEPARTAMENTO_DESC,
                    O.COSTE_DEPARTAMENTO  
                )

            WHEN NOT MATCHED BY SOURCE THEN  --LO QUE ESTÁ EN LA TABLA AFR PERO NO EN ORIGEN, CAMBIO LOS CAMPOS MODIFICADO_SN Y ELIMINADO_SN
                UPDATE SET 
                    AFR.MODIFICADO_SN=0,
                    AFR.ELIMINADO_SN=1
            ;

        COMMIT TRAN; --COMMIT PARA GUARDAR LOS CAMBIOS
    END TRY

    BEGIN CATCH --ES COMO EL EXCEPT EN PYTHON. SE EMPIEZA EL CATCH SI FALLA EL TRY (EN ESTE CASO HACE UN ROLLBACK)
        ROLLBACK TRAN;
    END CATCH

END;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.201

In [18]:
TRUNCATE TABLE PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO;
EXEC PRUEBAS_ALUMNOS.dbo.PR_AFR_REPARTO_COSTE_DPTO_2 201904, 201912; --DE ABRIL 19 A DIC 19
EXEC PRUEBAS_ALUMNOS.dbo.PR_AFR_REPARTO_COSTE_DPTO_2 201908, 202004; --DE AGOSTO 19 A ABRIL 20
EXEC PRUEBAS_ALUMNOS.dbo.PR_AFR_REPARTO_COSTE_DPTO_2 201904, 202004; --DE AGOSTO 19 A ABRIL 20

(270 rows affected)

(270 rows affected)

(270 rows affected)

(0 rows affected)

(390 rows affected)

(0 rows affected)

Total execution time: 00:00:00.204

In [19]:
SELECT DISTINCT MES_ID,
                MODIFICADO_SN,
                ELIMINADO_SN
FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO
ORDER BY MES_ID;

(9 rows affected)

Total execution time: 00:00:00.130

MES_ID,MODIFICADO_SN,ELIMINADO_SN
201904,0,0
201905,0,0
201906,0,0
201907,0,0
201908,0,0
201909,0,0
201910,0,0
201911,0,0
201912,0,0


In [20]:
SELECT COUNT(*) AS AFR FROM PRUEBAS_ALUMNOS.dbo.AFR_FACT_COSTE_DEPARTAMENTO;

(1 row affected)

Total execution time: 00:00:00.084

AFR
270


In [24]:
WITH CONSULTA_PIVOTAR AS (
    SELECT COMERCIAL_ID,
           MES_ID,
           DEPARTAMENTO_DESC,
           COSTE_DEPARTAMENTO
    FROM PRUEBAS_ALUMNOS.dbo.ICR_FACT_COSTE_DEPARTAMENTO
)

SELECT COMERCIAL_ID,
       MES_ID,
       VENTAS,
       MARKETING,
       RRHH,
       LOGISTICA,
       FINANCIERO
FROM CONSULTA_PIVOTAR

PIVOT (
    SUM(COSTE_DEPARTAMENTO)
    FOR DEPARTAMENTO_DESC IN (
        VENTAS,
        MARKETING,
        RRHH,
        LOGISTICA,
        FINANCIERO
    )
) AS PIVOTADA

(78 rows affected)

Total execution time: 00:00:00.115

COMERCIAL_ID,MES_ID,VENTAS,MARKETING,RRHH,LOGISTICA,FINANCIERO
COMERCIAL 115,201904,274.46,209.88,265.23,242.17,205.27
COMERCIAL 115,201905,275.31,179.55,265.73,292.07,184.34
COMERCIAL 115,201906,251.53,208.86,273.98,249.28,213.35
COMERCIAL 115,201907,289.10,193.53,250.87,274.76,188.75
COMERCIAL 115,201908,242.17,198.35,281.38,272.15,202.96
COMERCIAL 115,201909,190.43,276.99,220.11,225.06,284.41
COMERCIAL 115,201910,217.86,296.19,208.07,183.59,291.29
COMERCIAL 115,201911,213.03,266.28,207.95,235.85,273.89
COMERCIAL 115,201912,195.14,292.71,218.93,192.76,297.47
COMERCIAL 115,202001,223.67,292.49,204.01,189.26,287.57


In [1]:
SELECT TOP 10 * FROM NSLDINA.DBO.DIM_COMERCIAL

(6 rows affected)

Total execution time: 00:00:00.262

POSICIONABLE_ID,COMERCIAL_ID,SALARIO_BASE,COMISION_CLIENTE
115,COMERCIAL 115,900,12
116,COMERCIAL 116,1400,0
121,COMERCIAL 121,840,16
129,COMERCIAL 129,900,12
135,COMERCIAL 135,900,12
137,COMERCIAL 137,980,10


In [3]:
SELECT POSICIONABLE_ID,COMERCIAL_ID,CONCEPTO,IMPORTE 
FROM NSLDINA.DBO.DIM_COMERCIAL 
UNPIVOT (
    IMPORTE 
    FOR CONCEPTO IN (
        SALARIO_BASE,
        COMISION_CLIENTE
    )
) AS UPVT

(12 rows affected)

Total execution time: 00:00:00.229

POSICIONABLE_ID,COMERCIAL_ID,CONCEPTO,IMPORTE
115,COMERCIAL 115,SALARIO_BASE,900
115,COMERCIAL 115,COMISION_CLIENTE,12
116,COMERCIAL 116,SALARIO_BASE,1400
116,COMERCIAL 116,COMISION_CLIENTE,0
121,COMERCIAL 121,SALARIO_BASE,840
121,COMERCIAL 121,COMISION_CLIENTE,16
129,COMERCIAL 129,SALARIO_BASE,900
129,COMERCIAL 129,COMISION_CLIENTE,12
135,COMERCIAL 135,SALARIO_BASE,900
135,COMERCIAL 135,COMISION_CLIENTE,12


EJERCICIOS

In [ ]:
-- Obtenemos la venta total por provincia y producto


-- Calculamos los maximos para cada provincia


-- Obtenemos los productos que tienen el máximo de ventas para cada provincia

In [7]:
WITH TOTALES AS (
    SELECT DISTINCT ZONA_ID,PRODUCTO_ID, SUM(INGRESOS_EUR) OVER (PARTITION BY ZONA_ID,PRODUCTO_ID) AS VENTA_TOTAL
    FROM NEMINA.DBO.DATOS_BASE
),   

MAXIMOS AS (
    SELECT DISTINCT ZONA_ID, MAX(VENTA_TOTAL) OVER (PARTITION BY ZONA_ID) AS MAX_VENTA
    FROM TOTALES
)

SELECT V.ZONA_ID,V.PRODUCTO_ID,V.VENTA_TOTAL,M.MAX_VENTA
FROM TOTALES V 
INNER JOIN MAXIMOS M
    ON V.ZONA_ID=M.ZONA_ID AND V.VENTA_TOTAL=M.MAX_VENTA 
ORDER BY M.MAX_VENTA DESC

(4 rows affected)

Total execution time: 00:00:00.138

ZONA_ID,PRODUCTO_ID,VENTA_TOTAL,MAX_VENTA
Pontevedra,Setas,3999254.0000000000,3999254.0000000000
Coruña,Belleza,3458266.0000000000,3458266.0000000000
Ourense,Setas,1860857.0000000000,1860857.0000000000
Lugo,Algas,796240.0000000000,796240.0000000000


In [ ]:
-- Calculamos la venta total por provincia y cliente


-- Obtenemos el cliente y provincia junto con su venta, el ranking y al porcentaje de venta sobre el total

In [20]:
WITH VENTAS_PROVINCIA AS (
    SELECT DISTINCT C.PROVINCIA_ID, D.CLIENTE_ID, SUM(INGRESOS_EUR) OVER (PARTITION BY C.PROVINCIA_ID,D.CLIENTE_ID) AS VENTA_TOTAL
    FROM NEMINA.DBO.DATOS_BASE D
    INNER JOIN NEMINA.DBO.CLIENTES C 
        ON C.CLIENTE_ID = D.CLIENTE_ID
)


SELECT DISTINCT 
            RANK() OVER (PARTITION BY PROVINCIA_ID ORDER BY VENTA_TOTAL DESC) AS RANKING_PROVINCIA,
            CLIENTE_ID,
            PROVINCIA_ID,
            VENTA_TOTAL,
            VENTA_TOTAL/SUM(VENTA_TOTAL) OVER() *100 AS PCT_VENTA --No hay PARTITION BY porque aqui queremos obtener los Ingresos Totales

FROM VENTAS_PROVINCIA
ORDER BY VENTA_TOTAL DESC

(41 rows affected)

Total execution time: 00:00:00.109

RANKING_PROVINCIA,CLIENTE_ID,PROVINCIA_ID,VENTA_TOTAL,PCT_VENTA
1,27,27,1800610.0000000000,8.489200
1,25,32,1702679.0000000000,8.027500
2,28,27,1545420.0000000000,7.286100
1,23,36,1475847.0000000000,6.958000
1,24,15,1449001.0000000000,6.831500
2,22,36,1156044.0000000000,5.450300
3,21,27,1129908.0000000000,5.327100
3,18,36,939387.0000000000,4.428800
2,11,15,849125.0000000000,4.003300
3,26,15,820400.0000000000,3.867800


## **3.3 Coste total por departamento y temporada**

Usando la tabla que populamos con el procedimiento que creamos anteriormente, **PRUEBAS\_ALUMNOS.dbo.XXX\_FACT\_COSTE\_DEPARTAMENTO**, y la tabla **nSLDina.dbo.DIM\_TIEMPO**, obtener la temporada para la cual el coste de cada departamento es máximo ordenado por temporada y nombre de departamento.

In [10]:
SELECT * FROM NSLDINA.DBO.DIM_TIEMPO

(13 rows affected)

Total execution time: 00:00:00.084

MES_ID,TEMPORADA_ID
201904,Alta
201905,ALTA
201906,ALTA
201907,ALTA
201908,ALTA
201909,BAJA
201910,BAJA
201911,BAJA
201912,BAJA
202001,BAJA


In [20]:
WITH COSTES AS (
    SELECT DISTINCT T.TEMPORADA_ID,A.DEPARTAMENTO_ID, SUM(A.COSTE_DEPARTAMENTO) OVER (PARTITION BY T.TEMPORADA_ID,A.DEPARTAMENTO_ID) AS COSTE_MES_DPTO
    FROM PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO A 
    LEFT JOIN NSLDINA.DBO.DIM_TIEMPO T 
        ON T.MES_ID = A.MES_ID
),

ORDENADO AS (
    SELECT TEMPORADA_ID,DEPARTAMENTO_ID,COSTE_MES_DPTO, ROW_NUMBER() OVER (PARTITION BY DEPARTAMENTO_ID ORDER BY COSTE_MES_DPTO DESC) AS RN
    FROM COSTES
)

SELECT DISTINCT A.DEPARTAMENTO_DESC,O.TEMPORADA_ID,O.COSTE_MES_DPTO AS COSTE, O.RN FROM ORDENADO O
LEFT JOIN PRUEBAS_ALUMNOS.DBO.AFR_FACT_COSTE_DEPARTAMENTO A
    ON A.DEPARTAMENTO_ID = O.DEPARTAMENTO_ID
WHERE O.RN=1
ORDER BY COSTE DESC

(5 rows affected)

Total execution time: 00:00:00.132

DEPARTAMENTO_DESC,TEMPORADA_ID,COSTE,RN
RRHH,Alta,8795.76,1
VENTAS,Alta,8765.29,1
LOGISTICA,ALTA,8751.26,1
FINANCIERO,BAJA,7545.10,1
MARKETING,BAJA,7447.16,1


3.4 Pedidos por rangos con venta total de cada rango 

De la tabla Northwind.dbo.\[Order Details\] calculamos el número de pedidos y la venta total por cada uno de los siguientes rangos de venta, teniendo en cuenta que el importe de cada línea es UnitPrice \* Quantity \* (1 - Discount).  

0 - 99

. 100 - 999

. 1000 - 9999.

 10000 - +.

In [50]:
WITH IMP AS (
    SELECT DISTINCT ORDERID, SUM(UnitPrice * Quantity * (1 - Discount)) OVER (PARTITION BY ORDERID) AS IMPORTE FROM NORTHWIND.DBO.[ORDER DETAILS]
),

TABLA_RANGOS AS (
    SELECT
    ORDERID,
    CASE 
        WHEN IMPORTE>=0 AND IMPORTE<=99 THEN '0 - 99'
        WHEN IMPORTE>=100 AND IMPORTE<1000 THEN '100-999'
        WHEN IMPORTE>=1000 AND IMPORTE<9999 THEN '1000-9999'
        WHEN IMPORTE>=10000 THEN '10000+'
        END AS RANGOS,
    IMPORTE
    FROM IMP
)

SELECT RANGOS, COUNT(DISTINCT ORDERID) AS CONTEO, SUM(IMPORTE) AS IMPORTE_TOTAL, SUM(IMPORTE)/COUNT(DISTINCT ORDERID) AS IMPORTE_MEDIO_PEDIDO FROM TABLA_RANGOS
GROUP BY RANGOS

(4 rows affected)

Total execution time: 00:00:00.117

RANGOS,CONTEO,IMPORTE_TOTAL,IMPORTE_MEDIO_PEDIDO
0 - 99,8,601.0599966049194,75.13249957561493
100-999,111,52445.24513530731,472.47968590366946
1000-9999,118,265250.98498249054,2247.889703241445
10000+,2,21683.999976158142,10841.999988079071


## **3.5 Acumulado ventas producto y porcentaje sobre venta total**

Vamos a calcular el extracto acumulado de ventas de la tabla **nSLDina.dbo.FACT\_VENTA** por cada pedido para el producto **'Ensalada - 02'** y el mes **202002** junto con el acumulado de ventas totales y el porcentaje de venta del producto **'Ensalada - 02'** sobre las ventas totales.

In [63]:
WITH ACUMULADO AS (
    SELECT CLIENTE_ID,
           MES_ID,
           PRODUCTO_ID,
           CANTIDAD,
           SUM(CANTIDAD) OVER (PARTITION BY PRODUCTO_ID ORDER BY MES_ID,PRODUCTO_ID ROWS UNBOUNDED PRECEDING) AS ACUMULADO_PRODUCTO,
           SUM(CANTIDAD) OVER (ORDER BY MES_ID,PRODUCTO_ID ROWS UNBOUNDED PRECEDING) AS ACUMULADO_TOTAL 
    FROM NSLDINA.DBO.FACT_VENTA
)

SELECT MES_ID, 
       PRODUCTO_ID,
       CANTIDAD,
       ACUMULADO_PRODUCTO,
       ACUMULADO_TOTAL,
       CAST(ACUMULADO_PRODUCTO * 100.00 / ACUMULADO_TOTAL AS DECIMAL(5,2)) AS PCT_SOBRE_TOTAL
FROM ACUMULADO
WHERE MES_ID = 202002 AND 
      PRODUCTO_ID = 'ENSALADA - 02'
ORDER BY MES_ID, PRODUCTO_ID

(133 rows affected)

Total execution time: 00:00:00.324

MES_ID,PRODUCTO_ID,CANTIDAD,ACUMULADO_PRODUCTO,ACUMULADO_TOTAL,PCT_SOBRE_TOTAL
202002,Ensalada - 02,2,973,18561,5.24
202002,Ensalada - 02,1,974,18562,5.25
202002,Ensalada - 02,1,975,18563,5.25
202002,Ensalada - 02,1,976,18564,5.26
202002,Ensalada - 02,2,978,18566,5.27
202002,Ensalada - 02,1,979,18567,5.27
202002,Ensalada - 02,1,980,18568,5.28
202002,Ensalada - 02,1,981,18569,5.28
202002,Ensalada - 02,1,982,18570,5.29
202002,Ensalada - 02,2,984,18572,5.30


## **3.6 Cálculo de días de baja por mes.**

Este ejercicio vamos a realizarlo para dos tablas de baja con estructuras diferentes, para ambas tendremos que crear un calendario. Comenzaremos calculando los días de baja para el mes **202108** utilizando para ello la tabla **nSLDina.dbo.FACT\_BAJA\_RANGO**. En esta tabla tendremos almacenadas las bajas de cada comercial indicando el día de inicio de la baja y el día de fin de la misma.

In [86]:
SELECT DATEADD(DAY, ROW_NUMBER() OVER (ORDER BY CLIENTE_ID), CAST(CAST(MES_ID AS VARCHAR(6))+'01' AS DATE)) AS DIA_ID,MES_ID
FROM nSLDina.dbo.FACT_VENTA

(18214 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.623

DIA_ID,MES_ID
2019-08-02,201908
2019-08-03,201908
2019-08-04,201908
2019-08-05,201908
2019-08-06,201908
2019-08-07,201908
2019-08-08,201908
2019-09-09,201909
2019-09-10,201909
2019-09-11,201909


In [94]:
WITH CALENDARIO AS (
    SELECT DATEADD(DAY, ROW_NUMBER() OVER (ORDER BY CLIENTE_ID), CAST(CAST(MES_ID AS VARCHAR(6))+'01' AS DATE)) AS DIA_ID,MES_ID
    FROM nSLDina.dbo.FACT_VENTA
    ),

-- Realizamos el JOIN de la tabla de bajas con el calendario para dividir el rango en sus diferentes días.
DIAS_BAJA AS (
    SELECT COMERCIAL_ID,
        DIA_ID,
        MES_ID
    FROM nSLDina.dbo.FACT_BAJA_RANGO
        INNER JOIN CALENDARIO
            ON DIA_ID BETWEEN DIA_INICIO AND DIA_FIN
    )

SELECT COUNT(*) AS DIAS_BAJA_AGOSTO
FROM DIAS_BAJA
WHERE FORMAT(DIA_ID, 'yyyyMM') = 202108

(1 row affected)

Total execution time: 00:00:01.016

DIAS_BAJA_AGOSTO
24


In [108]:
WITH CALENDARIO AS (
    SELECT DATEADD(DAY, ROW_NUMBER() OVER (ORDER BY CLIENTE_ID), CAST(CAST(MES_ID AS VARCHAR(6))+'01' AS DATE)) AS DIA_ID,MES_ID
    FROM nSLDina.dbo.FACT_VENTA
    ),

EMPLEADOS_BAJA AS (
    SELECT DISTINCT COMERCIAL_ID
    FROM nSLDina.dbo.FACT_BAJA_ESTADO
    ),

TODOS_LOS_DIAS AS (
    SELECT C.DIA_ID,
           EB.COMERCIAL_ID,
           BAJA_SN,
           SUM(CASE WHEN FBE.COMERCIAL_ID IS NULL THEN 0 ELSE 1 END)  OVER (PARTITION BY EB.COMERCIAL_ID ORDER BY C.DIA_ID) AS GRUPO
    FROM CALENDARIO C
        CROSS JOIN EMPLEADOS_BAJA EB
        LEFT JOIN nSLDina.dbo.FACT_BAJA_ESTADO FBE
            ON C.DIA_ID = FBE.DIA_ID AND
                EB.COMERCIAL_ID = FBE.COMERCIAL_ID
    WHERE FORMAT(c.DIA_ID, 'yyyy') >= 2021
    ),

-- Ahora que tenemos los registros relacionados, vamos a cubrir los datos para los campos que no tienen valor.
TODOS_LOS_DIAS_SIN_NULOS AS (
    SELECT DIA_ID,
        COMERCIAL_ID,
        COALESCE(BAJA_SN, FIRST_VALUE(BAJA_SN) OVER (PARTITION BY COMERCIAL_ID, GRUPO ORDER BY DIA_ID)) AS BAJA_SN,
        GRUPO
    FROM TODOS_LOS_DIAS
    )

-- Una vez que tenemos todos los días con el estado informado, ya podremos hacer el recuento de días de baja filtrando por el mes 202108
SELECT COUNT(*) AS DIAS_BAJA_AGOSTO
FROM TODOS_LOS_DIAS_SIN_NULOS
WHERE BAJA_SN <>0 AND
      FORMAT(DIA_ID, 'yyyyMM') = 202108

(1 row affected)

Total execution time: 00:00:03.063

DIAS_BAJA_AGOSTO
16
